In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:
print("Hello World")

Hello World


In [3]:
train_df = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test_df = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')

In [4]:
train_df.info()
train_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8000 non-null   int64  
 1   ID          8000 non-null   int64  
 2   class       8000 non-null   object 
 3   20150720_N  7440 non-null   float64
 4   20150602_N  6800 non-null   float64
 5   20150517_N  7200 non-null   float64
 6   20150501_N  7040 non-null   float64
 7   20150415_N  7520 non-null   float64
 8   20150330_N  6880 non-null   float64
 9   20150314_N  7280 non-null   float64
 10  20150226_N  6640 non-null   float64
 11  20150210_N  7360 non-null   float64
 12  20150125_N  6960 non-null   float64
 13  20150109_N  7120 non-null   float64
 14  20141117_N  6720 non-null   float64
 15  20141101_N  7600 non-null   float64
 16  20141016_N  6560 non-null   float64
 17  20140930_N  7200 non-null   float64
 18  20140813_N  7440 non-null   float64
 19  20140626_N  6400 non-null  

Unnamed: 0       0
ID               0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64

In [5]:
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['class'])  # For model training

In [6]:
features = [col for col in train_df.columns if '_N' in col]

In [7]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(train_df[features])
X_test = imputer.transform(test_df[features])  # We'll use this later


In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)


In [9]:
y = train_df['label']
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)


In [10]:
model = LogisticRegression(max_iter=1000, multi_class='multinomial')
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [11]:
y_val_pred = model.predict(X_val)

print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))
print("Classification Report:\n", classification_report(y_val, y_val_pred, target_names=le.classes_))


Validation Accuracy: 0.8675
Confusion Matrix:
 [[  65   96    1    5    0    1]
 [  29 1191    8    3    0    1]
 [   1   23    6    9    0    0]
 [   7   12    1  114    0    0]
 [   1    4    0    0    1    0]
 [   1    2    0    7    0   11]]
Classification Report:
               precision    recall  f1-score   support

        farm       0.62      0.39      0.48       168
      forest       0.90      0.97      0.93      1232
       grass       0.38      0.15      0.22        39
  impervious       0.83      0.85      0.84       134
     orchard       1.00      0.17      0.29         6
       water       0.85      0.52      0.65        21

    accuracy                           0.87      1600
   macro avg       0.76      0.51      0.57      1600
weighted avg       0.85      0.87      0.85      1600



In [12]:
test_preds = model.predict(X_test_scaled)
test_classes = le.inverse_transform(test_preds)


In [13]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'class': test_classes
})
submission.to_csv('submission.csv', index=False)


In [14]:
# Drop 'Unnamed: 0' if it exists in test_df
if 'Unnamed: 0' in test_df.columns:
    test_df = test_df.drop(columns=['Unnamed: 0'])


In [15]:
X_test = test_df.drop(columns=['ID'])
X_test_imputed = imputer.transform(X_test)
X_test_scaled = scaler.transform(X_test_imputed)

test_preds = model.predict(X_test_scaled)
test_classes = le.inverse_transform(test_preds)


In [16]:
import shutil
shutil.move("submission.csv", "/kaggle/working/submission.csv")


'/kaggle/working/submission.csv'